In [4]:
import pandas as pd
import re
import os
from tqdm import tqdm
from transformers import pipeline
import huggingface_hub

In [ ]:
df = pd.read_csv(r'c:\Users\User\Documents\NIR3\files\problems_export_2024-06-06.csv')
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
df = df[df['ID'].apply(is_number)]
df

,ID,Внешний ID,Название,Район,Статус,Дата создания,Дата последнего обновления,Описание,Пользователь,ID Пользователя,Широта,Долгота,Адрес,Муниципальное образование,Причина обращения,Подкатегория,Категория
0,3411686,3988078,Несанкционированные надписи / объявления на со...,Московский,Рассмотрение,01.01.2022 02:35:43,02.01.2022 15:53:49,Объявление с рекламой услуг ХХХ на киоске,Symbian -.,/accounts/203021/,59.8506,30.3209,NaN,муниципальный округ Пулковский меридиан,Несанкционированные надписи / объявления на со...,Благоустройство,Сооружение
1,3411687,3988075,Несанкционированные надписи / объявления на ст...,Московский,Рассмотрение,01.01.2022 02:29:35,02.01.2022 15:54:21,"Граффити на доме, рядом с вывеской ""Адамас"".",Symbian -.,/accounts/203021/,59.8495,30.3205,"г.Санкт-Петербург, Московский проспект, дом 19...",муниципальный округ Пулковский меридиан,Несанкционированные надписи / объявления на ст...,Благоустройство,Дом
2,3411688,3988073,Несанкционированные надписи / объявления на ст...,Московский,Рассмотрение,01.01.2022 02:21:05,02.01.2022 17:05:34,"Граффити теггинг на дверях, откуда мусор забир...",Symbian -.,/accounts/203021/,59.8413,30.3262,"г.Санкт-Петербург, Московское шоссе, дом 2, ли...",муниципальный округ Звездное,Несанкционированные надписи / объявления на ст...,Благоустройство,Дом
3,3411773,3988242,Несанкционированные надписи / объявления на ст...,Петроградский,Рассмотрение,01.01.2022 11:40:36,02.01.2022 17:07:13,надписи,Дмитрий С.,/accounts/23096/,59.9692,30.3087,"г.Санкт-Петербург, Каменноостровский проспект,...",муниципальный округ Аптекарский остров,Несанкционированные надписи / объявления на ст...,Благоустройство,Дом
4,3411774,3988239,"Неисправно освещение, опора освещения повреждена",Красногвардейский,Рассмотрение,01.01.2022 11:37:07,02.01.2022 17:06:51,"Неисправно освещение (2 фонаря, у тротуара, юж...",Петр Т.,/accounts/237000/,59.9689,30.4269,NaN,муниципальный округ Полюстрово,"Неисправно освещение, опора освещения повреждена",Благоустройство,Территория Санкт-Петербурга
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104345,3693401,4631183,Разукомплектованное транспортное средство,Колпинский,Завершено: Автоматически,11.10.2022 10:01:24,07.12.2022 01:32:15,машина стоит больше 3 лет без движения. салон ...,Юлия И.,/accounts/243342/,59.7360,30.6201,NaN,город Колпино,Разукомплектованное транспортное средство,Благоустройство,Территория Санкт-Петербурга
104346,3693402,4631181,Мусор на внутридворовой территории,Невский,Завершено: Автоматически,11.10.2022 10:00:07,04.11.2022 01:30:47,Мусор на газоне.,Людмила К.,/accounts/127635/,59.8301,30.5138,NaN,муниципальный округ Рыбацкое,Мусор на внутридворовой территории,Благоустройство,Двор
104347,3693403,4631178,Отсутствие или несоблюдение графика уборки под...,Невский,Получен ответ,11.10.2022 09:58:19,26.10.2022 10:21:28,Не соблюдается график уборки влажного подметан...,Анна М.,/accounts/262840/,59.8961,30.4184,"г.Санкт-Петербург, улица Ольминского, дом 22, ...",муниципальный округ Невская застава,Отсутствие или несоблюдение графика уборки под...,Содержание МКД,Дом
104348,3693404,4631176,Мусор на газонах вдоль проезжей части,Невский,Завершено: Автоматически,11.10.2022 09:56:55,03.11.2022 01:31:05,Мусор на газоне вдоль проезжей части.,Людмила К.,/accounts/127635/,59.8298,30.5154,NaN,муниципальный округ Рыбацкое,Мусор на газонах вдоль проезжей части,Благоустройство,Улица


In [2]:
df.rename(columns={
    'Дата создания': 'date',
    'ID': 'id',
    'Подкатегория': 'cats',
    'Описание' : 'text',
    'Адрес' : 'loc',
    'Район': 'loc_dist'
}, inplace=True)

df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M:%S').dt.strftime('%Y-%m-%d')
df


,id,Внешний ID,Название,loc_dist,Статус,date,Дата последнего обновления,text,Пользователь,ID Пользователя,Широта,Долгота,loc,Муниципальное образование,Причина обращения,cats,Категория
0,3411686,3988078,Несанкционированные надписи / объявления на со...,Московский,Рассмотрение,2022-01-01,02.01.2022 15:53:49,Объявление с рекламой услуг ХХХ на киоске,Symbian -.,/accounts/203021/,59.8506,30.3209,NaN,муниципальный округ Пулковский меридиан,Несанкционированные надписи / объявления на со...,Благоустройство,Сооружение
1,3411687,3988075,Несанкционированные надписи / объявления на ст...,Московский,Рассмотрение,2022-01-01,02.01.2022 15:54:21,"Граффити на доме, рядом с вывеской ""Адамас"".",Symbian -.,/accounts/203021/,59.8495,30.3205,"г.Санкт-Петербург, Московский проспект, дом 19...",муниципальный округ Пулковский меридиан,Несанкционированные надписи / объявления на ст...,Благоустройство,Дом
2,3411688,3988073,Несанкционированные надписи / объявления на ст...,Московский,Рассмотрение,2022-01-01,02.01.2022 17:05:34,"Граффити теггинг на дверях, откуда мусор забир...",Symbian -.,/accounts/203021/,59.8413,30.3262,"г.Санкт-Петербург, Московское шоссе, дом 2, ли...",муниципальный округ Звездное,Несанкционированные надписи / объявления на ст...,Благоустройство,Дом
3,3411773,3988242,Несанкционированные надписи / объявления на ст...,Петроградский,Рассмотрение,2022-01-01,02.01.2022 17:07:13,надписи,Дмитрий С.,/accounts/23096/,59.9692,30.3087,"г.Санкт-Петербург, Каменноостровский проспект,...",муниципальный округ Аптекарский остров,Несанкционированные надписи / объявления на ст...,Благоустройство,Дом
4,3411774,3988239,"Неисправно освещение, опора освещения повреждена",Красногвардейский,Рассмотрение,2022-01-01,02.01.2022 17:06:51,"Неисправно освещение (2 фонаря, у тротуара, юж...",Петр Т.,/accounts/237000/,59.9689,30.4269,NaN,муниципальный округ Полюстрово,"Неисправно освещение, опора освещения повреждена",Благоустройство,Территория Санкт-Петербурга
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104345,3693401,4631183,Разукомплектованное транспортное средство,Колпинский,Завершено: Автоматически,2022-10-11,07.12.2022 01:32:15,машина стоит больше 3 лет без движения. салон ...,Юлия И.,/accounts/243342/,59.7360,30.6201,NaN,город Колпино,Разукомплектованное транспортное средство,Благоустройство,Территория Санкт-Петербурга
104346,3693402,4631181,Мусор на внутридворовой территории,Невский,Завершено: Автоматически,2022-10-11,04.11.2022 01:30:47,Мусор на газоне.,Людмила К.,/accounts/127635/,59.8301,30.5138,NaN,муниципальный округ Рыбацкое,Мусор на внутридворовой территории,Благоустройство,Двор
104347,3693403,4631178,Отсутствие или несоблюдение графика уборки под...,Невский,Получен ответ,2022-10-11,26.10.2022 10:21:28,Не соблюдается график уборки влажного подметан...,Анна М.,/accounts/262840/,59.8961,30.4184,"г.Санкт-Петербург, улица Ольминского, дом 22, ...",муниципальный округ Невская застава,Отсутствие или несоблюдение графика уборки под...,Содержание МКД,Дом
104348,3693404,4631176,Мусор на газонах вдоль проезжей части,Невский,Завершено: Автоматически,2022-10-11,03.11.2022 01:31:05,Мусор на газоне вдоль проезжей части.,Людмила К.,/accounts/127635/,59.8298,30.5154,NaN,муниципальный округ Рыбацкое,Мусор на газонах вдоль проезжей части,Благоустройство,Улица


In [3]:

from shapely.geometry import Point


def create_point(row):
    return Point(row['Долгота'], row['Широта'])

df['geometry'] = df.apply(create_point, axis=1)

df.drop(columns=['Широта', 'Долгота', 'Внешний ID', 'Название', 'Статус', 'Дата последнего обновления', 'Пользователь', 'ID Пользователя', 'Муниципальное образование', 'Причина обращения', 'Категория'], inplace=True)

df['source'] = 'НСПБ'
df

,id,loc_dist,date,text,loc,cats,geometry,source
0,3411686,Московский,2022-01-01,Объявление с рекламой услуг ХХХ на киоске,NaN,Благоустройство,POINT (30.3209 59.8506),НСПБ
1,3411687,Московский,2022-01-01,"Граффити на доме, рядом с вывеской ""Адамас"".","г.Санкт-Петербург, Московский проспект, дом 19...",Благоустройство,POINT (30.3205 59.8495),НСПБ
2,3411688,Московский,2022-01-01,"Граффити теггинг на дверях, откуда мусор забир...","г.Санкт-Петербург, Московское шоссе, дом 2, ли...",Благоустройство,POINT (30.3262 59.8413),НСПБ
3,3411773,Петроградский,2022-01-01,надписи,"г.Санкт-Петербург, Каменноостровский проспект,...",Благоустройство,POINT (30.3087 59.9692),НСПБ
4,3411774,Красногвардейский,2022-01-01,"Неисправно освещение (2 фонаря, у тротуара, юж...",NaN,Благоустройство,POINT (30.4269 59.9689),НСПБ
...,...,...,...,...,...,...,...,...
104345,3693401,Колпинский,2022-10-11,машина стоит больше 3 лет без движения. салон ...,NaN,Благоустройство,POINT (30.6201 59.736),НСПБ
104346,3693402,Невский,2022-10-11,Мусор на газоне.,NaN,Благоустройство,POINT (30.5138 59.8301),НСПБ
104347,3693403,Невский,2022-10-11,Не соблюдается график уборки влажного подметан...,"г.Санкт-Петербург, улица Ольминского, дом 22, ...",Содержание МКД,POINT (30.4184 59.8961),НСПБ
104348,3693404,Невский,2022-10-11,Мусор на газоне вдоль проезжей части.,NaN,Благоустройство,POINT (30.5154 59.8298),НСПБ


In [5]:
def clean_text(text):

    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  
    text = re.sub(r'[A-Za-z]', '', text)
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'\n', ' ', text)
    return text

df = df[df['text'].str.len() >= 10]
df['text'] = df['text'].apply(clean_text)

C:\Users\User\AppData\Local\Temp\ipykernel_15984\3563909181.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_text)


In [6]:
repository_id = 'Glebosol/spb_city_prob_class'
classifier = pipeline("sentiment-analysis", model=repository_id, tokenizer=repository_id, device=0)
MAX_LENGTH = 2048 

Device set to use cuda:0


In [7]:
def classify_text(text):
    text = text[:MAX_LENGTH]
    result = classifier(text)[0]
    return result['label'], result['score']


tqdm.pandas(desc="Классификация текста")
df[['label_cat', 'score_cat']] = df['text'].progress_apply(lambda x: pd.Series(classify_text(x)))

Классификация текста: 100%|██████████| 100526/100526 [05:57<00:00, 281.37it/s]
C:\Users\User\AppData\Local\Temp\ipykernel_15984\932756120.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['label_cat', 'score_cat']] = df['text'].progress_apply(lambda x: pd.Series(classify_text(x)))
C:\Users\User\AppData\Local\Temp\ipykernel_15984\932756120.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['label_cat', 'score_cat']] = df['text'].progress_apply(lambda x: pd.Series(classify_text(x)))


In [9]:
df['label_cat'].value_counts()

label_cat
ЖКХ,                41387
Дороги,             21587
Отходы,             12969
Энергетика           9112
Благоустройство,     9099
Транспорт,           4044
Безопасность,        1038
Строительство,        977
Здравоохранение,      157
Образование,          111
СоцЗащита,             45
Name: count, dtype: int64

In [11]:
df.to_csv('NSPB.csv', sep=';', index=False)